##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

# Кастомные колбеки в Keras
Кастомный колбек это мощный инструмент для настройки поведения модели Keras во время обучения, оценки или вывода, включая чтение/изменение модели Keras. Примеры включают `tf.keras.callbacks.TensorBoard` где процесс обучения и результаты могут быть экспортированы и визуализированы в TensorBoard, или `tf.keras.callbacks.ModelCheckpoint` где модель автоматически сохраняется во время обучения, и т.д. В этом руководстве вы узнаете, что такое колбек Keras, когда он будет вызван, что он может делать, и как вы можете построить свой колбек. Ближе к концу руководства будет несколько демонстраций создания пары простых колбек-приложений, чтобы помочь вам начать делать собственый колбек.

## Установка

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version существует только в Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## Введение в колбеки Keras
В Keras, `Callback` это базовый класс python предназначенный для наследования и обеспечивающий определенную функциональность. Класс `Callback` имеет предопределнный набор методов, вызываемых на различных этапах обучения (включая начало и конец пакета/эпохи), тестирования и прогнозирования. Колбеки используются для того, чтобы получить представлениее о внутренних состояниях и статистике модели во время обучения. Вы можете передать список колбеков (в качестве ключевого слова аргумента `callbacks`) любому из методов `tf.keras.Model.fit()`, `tf.keras.Model.evaluate()` и `tf.keras.Model.predict()`. Методы колбеков будут вызываться на разных этапах обучения/оценки/вывода.

Для начала давайте импортируем tensorflow и определим простую Sequential модель Keras:

In [ ]:
# Определим модель Keras чтобы добавить в нее колбеки
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

Затем загрузим данные MNIST из Keras datasets API для обучения и тестирования:

In [ ]:
# Загрузим данные MNIST и предобработаем их
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

Теперь определим простой колбек который будет просто отслеживать начало и конец каждого пакета данных. Во время этих вызовов он будет печатать индекс текущего пакета.

In [ ]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_begin(self, batch, logs=None):
    print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_end(self, batch, logs=None):
    print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

Передача колбека в метод модели, например `tf.keras.Model.fit()` гарантирует, что колбек будет вызываться на тех этапах, которые мы в нем определили. Например в примере выше это будет начало/конец тренировочного и тестового пакета(batch).

In [ ]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

## Методы Model работающие с колбеками
Пользователи могут передавать список колбеков в следующие методы `tf.keras.Model`:
#### [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator)
Обучает модель за фиксированное количество эпох (итерации по датасету, или данные полученные по-пакетно с помощью генератора Python).
#### [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate_generator)
Оценивает модель для имеющихся данных или генератора данных. Выводит значения потерь и метрик во время оценки.
#### [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict_generator)
Генерирует предсказания для входных данных или генератора данных.


In [ ]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

## Обзор методов колбеков


### Общие методы для обучения/тестирования/предсказания
Для обучения, тестирования и предсказания предоставляются следующие методы для переопределения.
#### `on_(train|test|predict)_begin(self, logs=None)`
Вызывается в начале `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_end(self, logs=None)`
Вызывается в конце `fit`/`evaluate`/`predict`.

### Batch-level методы для обучения/тестирования/предсказания
#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
Вызывается непосредственно перед обработкой пакета во время обучения/тестирования/предсказания. С этим методом, `logs` это словарь с ключами `batch` и `size`, представляющие номер текущего пакета и размер пакета.
#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`
Вызывается в конце обучения/тестирования/предсказания на пакете. В этом методе `logs` это словарь содержащий результаты метрик.

### Epoch-level  методы(используются только в процессе обучения)
#### on_epoch_begin(self, epoch, logs=None)
Вызывается в начале эпохи во время обучения.
#### on_epoch_end(self, epoch, logs=None)
Вызывается в конце эпохи во время обучения.


### Использование словаря `logs`
Словарь `logs` содержит величину потерь(loss) и все метрики в конце пакета или эпохи. Пример ниже включает величину потерь(loss) и среднеквадратичную ошибку.

In [ ]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('Средние потери за эпоху {} равны {:7.2f}, а среднеквадратичная ошибка равна {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

Аналогично можно использовать колбеки в вызове `evaluate()`.

In [ ]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

## Примеры колбек-модулей Keras
Следующий раздел поможет вам в создании простых  Callback модулей.

### Ранняя остановка при минимальном значении функции потерь
Первый пример демонстрирует создание `Callback`а который останавливает обучение Keras когда достигнута минимумальная величина потерь, путем изменения аргумента `model.stop_training` (булево значение). Опционально пользователь может использовать аргумент `patience`, чтобы указать сколько дополнительных эпох обучаться до полной остановки при получении минимального значения потерь.

`tf.keras.callbacks.EarlyStopping` обеспечиваает более полную и общую реализацию.

In [ ]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Остановить обучение, когда loss на минимуме, т.е. loss прекратил уменьшаться.

  Аргументы:
      patience: Количество эпох ожидания после достижения минимума. Если столько
      эпох нет улучшения, обучение останавливается.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights для хранения весов на которых достигнут минимум потерь.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # Количество эпох ожидания после достижения минимума.
    self.wait = 0
    # Эпоха на которой остановилось обучение.
    self.stopped_epoch = 0
    # Устанавливаем self.best равным бесконечности.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Записать лучшие веса если текущая величина потерь меньше значения в self.best.
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Восстановление весов модели с конца лучшей эпохи.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Эпоха %05d: ранняя остановка' % (self.stopped_epoch + 1))

In [ ]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

### Планирование скорости обучения

При обучении модели часто используют изменение скорости обучения по мере того, как проходит больше эпох.

Примечание: это лишь реализация примера, для более глубокого понимания смотрите `callbacks.LearningRateScheduler` и `keras.optimizers.schedules`.

In [ ]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Планировщик скорости обучения, устанавливающий скорость в соответствии с расписанием.

  Аргументы:
      schedule: функция которая получает на вход индекс эпохи
          (целое число, индексируемое с 0) и текущую скорость обучения
          и возвращает на выходе новую скорость обучения(float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Получаем текущую скорость обучения из оптимизатора модели.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Вызываем функцию расписания, чтобы получить запланированную скорость обучения.
    scheduled_lr = self.schedule(epoch, lr)
    # Устанавливаем новое значение в оптимизатор до начала новой эпохи
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [ ]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """Вспомогательная функция для получения запланированной скорости обучения на основе порядкового номера эпохи."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])

### Стандартные колбеки Keras
Узнать больше о колбеках Keras вы можете [прочитав полную документацию по API](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks). Там представлены такие колбеки как логирование в CSV, сохранение модели, визуализацию на TensorBoard и многое другое.